In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
train_df_path = "~/data/train.csv"
train_df = pd.read_csv(train_df_path)
train_df.head()

,idx,class,conversation
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...


In [3]:
train_df.groupby(by=['class']).count()

,idx,conversation
class,,
갈취 대화,981,981
기타 괴롭힘 대화,1094,1094
직장 내 괴롭힘 대화,979,979
협박 대화,896,896


### Label Encoding

In [4]:
label_encode = {
    "협박 대화" : 0,
    "갈취 대화" : 1,
    "직장 내 괴롭힘 대화" : 2,
    "기타 괴롭힘 대화" : 3,   
}
train_df['encoded_label'] = train_df['class'].map(label_encode)

In [5]:
train_df['encoded_label']

0       0
1       0
2       3
3       1
4       1
       ..
3945    3
3946    1
3947    2
3948    1
3949    2
Name: encoded_label, Length: 3950, dtype: int64

### Spliting data into training and validation set

In [6]:
train_texts = train_df['conversation'].to_list()
train_labels = train_df['encoded_label'].to_list()

In [7]:
from sklearn.model_selection import train_test_split

# Stratified Split Train and Validation data 
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.2, random_state=1004, stratify=train_labels)

# 2. Tokenizing the text
## Load Tokenizer and Tokenizing

In [8]:
MODEL_PATH = "klue/bert-base"

In [9]:
from transformers import BertTokenizerFast

# Load Tokenizer 
tokenizer = BertTokenizerFast.from_pretrained(MODEL_PATH)

# Tokenizing
# dict_keys(['input_ids', 'token_type_ids', 'attention_mask']) 이런식으로 
train_encodings = tokenizer(train_texts, truncation=True, padding=True) 
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

In [10]:
print(dict(val_encodings).keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])


# 3. Creating a Dataset Object for Tensorflow

In [11]:
import tensorflow as tf

# trainset-set
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))

# validation-set
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
))

# 4. Fine-Tuning BERT
## 4.1 Using Native Tensorflow pipeline
### Load Pretrained Model

In [12]:
from transformers import TFBertForSequenceClassification
from tensorflow.keras import optimizers
import tensorflow_addons as tfa

num_labels = len(label_encode)
print(num_labels)
# TODO : from_pt=False 혹은 없이 해보기
# from_pt – (optional) boolean, default False: Load the model weights from a PyTorch state_dict save file (see docstring of pretrained_model_name_or_path argument).
model = TFBertForSequenceClassification.from_pretrained(MODEL_PATH, num_labels=num_labels, from_pt=True)

optimizer = tfa.optimizers.RectifiedAdam(lr=5.0e-5, total_steps = 2344*4, warmup_proportion=0.1, min_lr=1e-5, epsilon=1e-08, clipnorm=1.0)
model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=['accuracy'])

4


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.9/site-packages/keras/optimizer_v2/optimize

In [13]:
model.compute_loss

<bound method TFSequenceClassificationLoss.compute_loss of <transformers.models.bert.modeling_tf_bert.TFBertForSequenceClassification object at 0x7f8028efc790>>

In [14]:
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint

callback_earlystopping = EarlyStopping(
    monitor="val_accuracy", 
    min_delta=0.001, # the threshold that triggers the termination (acc should at least improve 0.001)
    patience=2)

callback_learningrate_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=2,
    verbose=0,
    mode='auto',
    min_delta=0.0001,
    cooldown=0,
    min_lr=0,
)


callback_modelcheckpoint = ModelCheckpoint(
    filepath = "BERT_BestModel.keras",
    monitor="vall_accuracy",
    save_best_only=True,
)

callback_list = [callback_earlystopping, callback_learningrate_scheduler, callback_modelcheckpoint]

model.fit(
    train_dataset.shuffle(1000).batch(8), epochs=10, batch_size=8,
    validation_data=val_dataset.shuffle(1000).batch(16),
    callbacks = callback_list
)

Epoch 1/10
395/395 [==============================] - 447s 996ms/step - loss: 1.0944 - accuracy: 0.5266 - val_loss: 0.4741 - val_accuracy: 0.8557
Epoch 2/10
395/395 [==============================] - 392s 993ms/step - loss: 0.3532 - accuracy: 0.8801 - val_loss: 0.4548 - val_accuracy: 0.8684
Epoch 3/10
395/395 [==============================] - 392s 993ms/step - loss: 0.1978 - accuracy: 0.9383 - val_loss: 0.3958 - val_accuracy: 0.8911
Epoch 4/10
395/395 [==============================] - 392s 994ms/step - loss: 0.1103 - accuracy: 0.9646 - val_loss: 0.4749 - val_accuracy: 0.8924
Epoch 5/10
395/395 [==============================] - 392s 993ms/step - loss: 0.0599 - accuracy: 0.9842 - val_loss: 0.5986 - val_accuracy: 0.8646
Epoch 6/10
395/395 [==============================] - 392s 993ms/step - loss: 0.0227 - accuracy: 0.9943 - val_loss: 0.4862 - val_accuracy: 0.8949
Epoch 7/10
395/395 [==============================] - 392s 993ms/step - loss: 0.0119 - accuracy: 0.9972 - val_loss: 0.4960 -

### Saving the model and tokenizer

In [15]:
MODEL_NAME = 'fine-tuned-klue-bert-base'
MODEL_SAVE_PATH = os.path.join("_model", MODEL_NAME) # change this to your preferred location

if os.path.exists(MODEL_SAVE_PATH):
    print(f"{MODEL_SAVE_PATH} -- Folder already exists \n")
else:
    os.makedirs(MODEL_SAVE_PATH, exist_ok=True)
    print(f"{MODEL_SAVE_PATH} -- Folder create complete \n")

# save tokenizer, model
model.save_pretrained(MODEL_SAVE_PATH)
tokenizer.save_pretrained(MODEL_SAVE_PATH)

_model/fine-tuned-klue-bert-base -- Folder already exists 



('_model/fine-tuned-klue-bert-base/tokenizer_config.json',
 '_model/fine-tuned-klue-bert-base/special_tokens_map.json',
 '_model/fine-tuned-klue-bert-base/vocab.txt',
 '_model/fine-tuned-klue-bert-base/added_tokens.json',
 '_model/fine-tuned-klue-bert-base/tokenizer.json')

# 6. Loading the saved Model and Prediction

In [16]:
from transformers import TextClassificationPipeline

# Load Fine-tuning model
loaded_tokenizer = BertTokenizerFast.from_pretrained(MODEL_SAVE_PATH)
loaded_model = TFBertForSequenceClassification.from_pretrained(MODEL_SAVE_PATH)

text_classifier = TextClassificationPipeline(
    tokenizer=loaded_tokenizer, 
    model=loaded_model, 
    framework='tf',
    return_all_scores=True
)

Some layers from the model checkpoint at _model/fine-tuned-klue-bert-base were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at _model/fine-tuned-klue-bert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [17]:
test_df = pd.DataFrame(columns=['file_name', 'class'])
test_df.head()

,file_name,class


In [18]:
test_data = pd.read_csv('./test_data.csv', index_col=0)
test_data

,text
t_000,아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...
t_001,우리팀에서 다른팀으로 갈 사람 없나? 그럼 영지씨가 가는건 어때? 네? 제가요? ...
t_002,너 오늘 그게 뭐야 네 제가 뭘 잘못했나요.? 제대로 좀 하지 네 똑바로 좀 하지 ...
t_004,아무튼 앞으로 니가 내 와이파이야. .응 와이파이 온. 켰어. 반말? 주인님이라고도...
t_005,그러니까 빨리 말해. 선생님 제발 살려주십시오. 비밀번호 틀릴 때마다 손톱 하나씩...
...,...
t_495,미나씨 휴가 결제 올리기 전에 저랑 상의하라고 말한거 기억해요? 네 합니다. 보고서...
t_496,교수님 제 논문에 제 이름이 없나요? 아 무슨 논문말이야? 지난 번 냈던 논문이...
t_497,야 너 네 저요? 그래 너 왜요 돈좀 줘봐 돈 없어요 돈이 왜 없어 지갑은 폼이...
t_498,야 너 빨리 안 뛰어와? 너 이 환자 제대로 봤어 안 봤어 어제 저녁부터 계속 보다...


In [19]:
import json

with open('/aiffel/data/test.json', 'r') as f:
    test_json = json.load(f)
    
# test_json

In [20]:
from tqdm.auto import tqdm
answer_dict = {}
for file_name, text in tqdm(test_json.items()):
    preds_list = text_classifier(text['text'])[0]
    best_label = int(sorted(preds_list, key=lambda x : x['score'])[-1]['label'].split('_')[-1])
    answer_dict[file_name] = best_label
          
answer_dict

  0%|          | 0/400 [00:00<?, ?it/s]

{'t_000': 1,
 't_001': 2,
 't_002': 2,
 't_004': 3,
 't_005': 0,
 't_006': 0,
 't_007': 3,
 't_009': 1,
 't_010': 0,
 't_012': 2,
 't_014': 2,
 't_015': 0,
 't_018': 0,
 't_019': 3,
 't_020': 0,
 't_021': 3,
 't_022': 3,
 't_023': 1,
 't_024': 1,
 't_025': 2,
 't_028': 2,
 't_030': 1,
 't_031': 0,
 't_033': 1,
 't_034': 3,
 't_035': 3,
 't_036': 3,
 't_037': 0,
 't_038': 0,
 't_039': 1,
 't_040': 0,
 't_041': 1,
 't_042': 2,
 't_043': 1,
 't_045': 2,
 't_046': 0,
 't_047': 3,
 't_049': 0,
 't_050': 1,
 't_051': 3,
 't_052': 1,
 't_053': 2,
 't_054': 2,
 't_055': 3,
 't_056': 3,
 't_058': 3,
 't_059': 3,
 't_060': 1,
 't_061': 3,
 't_062': 0,
 't_063': 3,
 't_064': 2,
 't_065': 0,
 't_066': 3,
 't_067': 3,
 't_069': 1,
 't_071': 2,
 't_072': 3,
 't_073': 2,
 't_074': 0,
 't_076': 2,
 't_077': 1,
 't_078': 2,
 't_081': 0,
 't_083': 0,
 't_084': 3,
 't_085': 2,
 't_086': 0,
 't_088': 1,
 't_089': 2,
 't_091': 2,
 't_092': 0,
 't_093': 3,
 't_095': 2,
 't_097': 2,
 't_098': 1,
 't_099': 0,

In [21]:
for key, value in answer_dict.items():
    test_df = test_df.append({'file_name': key, 'class': value}, ignore_index=True)

In [22]:
# test_df.to_csv("HanBERT.csv")  # accuracy - 0.9125
# test_df.to_csv("HanBERT_ver2.csv")  # accuracy - 0.875
test_df.to_csv("HanBERT_ver3.csv")